# Data extraction and areal interpolation

This is the first notebook for the first phase of APPG-LBA project of Geographic Data Science Lab, University of Liverpool.

For the best performance, it uses a custom environment based on Docker container `darribas/gds_py:6.0alpha1` updated to use pygeos=0.9, master version of GeoPandas (as of 23/02/2021 - to be 0.9.0 release) and custom development branch of tobler, which implements required categorical areal interpolation (https://github.com/martinfleis/tobler/tree/categorical).

The notebook downloads required data (LSOA geography, CORINE Land Cover) and interpolates CLC to LSOA.

In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
import tobler
import zipfile
import fiona
import pygeos
from shapely.geometry import box

from download import download
from tobler.area_weighted import area_interpolate

In [2]:
tobler.__version__, gpd.__version__, pygeos.__version__

('0.7.0', '0.8.0+113.g4444f1a', '0.9')

## Download data

All data are from open sources, either geoportal.statistics.gov.uk (LSOA) or land.copernicus.eu (CLC).

### LSOA

Lower Layer Super Output Areas (December 2011) Boundaries Full Clipped (BFC) EW V3

Source: https://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-areas-december-2011-boundaries-full-clipped-bfc-ew-v3?geometry=-33.813%2C48.013%2C29.468%2C57.298

In [3]:
lsoa_path = download("https://opendata.arcgis.com/datasets/1f23484eafea45f98485ef816e4fee2d_0.zip?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D",
                     "../../data/lsoa", kind='zip')

Creating data folder...

file_sizes: 222MB [00:06, 33.6MB/s]                                             
Extracting zip file...
Successfully downloaded / unzipped to ../../data/lsoa


### CORINE Land Cover

Downloading CORINE Land Cover requires login, after which we can generate temporary link. The links used below will not work anymore and to reproduce the code, one needs to request a new link from the Copernicus portal.

https://land.copernicus.eu/pan-european/corine-land-cover

Data for all years covering England (2000, 2006, 2012, 2018) are downloaded and unzipped. The selected format is GPKG vector layer.

In [4]:
clc2000 = download("https://land.copernicus.eu/land-files/862d37e5a0aeacfbf09285152a2f34c0cb8e84e8.zip",
                  "../../data/clc/2000", kind='zip')

Creating data folder...

file_sizes: 100%|██████████████████████████| 2.34G/2.34G [00:47<00:00, 49.3MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ../../data/clc/2000


In [12]:
with zipfile.ZipFile(clc2000 + "/u2006_clc2000_v2020_20u1_geoPackage.zip", 'r') as zip_ref:
    zip_ref.extractall("../../data/clc/2000")

In [5]:
clc2006 = download("https://land.copernicus.eu/land-files/2962e747e238a4045503652b61f986675122c45c.zip",
                  "../../data/clc/2006", kind='zip')

Creating data folder...

file_sizes: 100%|██████████████████████████| 3.27G/3.27G [00:52<00:00, 62.5MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ../../data/clc/2006


In [13]:
with zipfile.ZipFile(clc2006 + "/u2012_clc2006_v2020_20u1_geoPackage.zip", 'r') as zip_ref:
    zip_ref.extractall("../../data/clc/2006")

In [6]:
clc2012 = download("https://land.copernicus.eu/land-files/2fd769e3afcef57f2e47ae4ea566140d226e78eb.zip",
                  "../../data/clc/2012", kind='zip')

Creating data folder...

file_sizes: 100%|██████████████████████████| 3.78G/3.78G [01:48<00:00, 34.8MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ../../data/clc/2012


In [14]:
with zipfile.ZipFile(clc2012 + "/u2018_clc2012_v2020_20u1_geoPackage.zip", 'r') as zip_ref:
    zip_ref.extractall("../../data/clc/2012")

In [7]:
clc2018 = download("https://land.copernicus.eu/land-files/1cefc5e87d3cecf0f145fc4d0823668669f51423.zip",
                  "../../data/clc/2018", kind='zip')

Creating data folder...

file_sizes: 100%|██████████████████████████| 3.76G/3.76G [01:51<00:00, 33.7MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ../../data/clc/2018


In [15]:
with zipfile.ZipFile(clc2018 + "/u2018_clc2018_v2020_20u1_geoPackage.zip", 'r') as zip_ref:
    zip_ref.extractall("../../data/clc/2018")

## Interpolate categories

Having all data downloaded, we can interpolate CLC classification onto LSOA geoemtry.

In [4]:
lsoa_path = "../../data/lsoa"
lsoa = gpd.read_file(lsoa_path)

Since CLC data cover the whole Europe, we will filter them on read by a bounding box of LSOA GeoDataFrame.

In [5]:
bbox = gpd.GeoSeries([box(*lsoa.total_bounds)], crs=lsoa.crs).to_crs(3035)

### 2000

Check layers to make sure we're loading the correct one.

In [6]:
path2000 = "../../data/clc/2000/u2006_clc2000_v2020_20u1_geoPackage/DATA/U2006_CLC2000_V2020_20u1.gpkg"
lyr = fiona.listlayers(path2000)
lyr

['U2006_CLC2000_V2020_20u1']

Read data within the bounding box.

In [7]:
clc2000 = gpd.read_file(path2000, bbox=bbox)

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:565: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Reproject to CRS of LSOA.

In [8]:
clc2000 = clc2000.to_crs(lsoa.crs)

In [9]:
clc2000.columns

Index(['code_00', 'ID', 'Remark', 'Area_Ha', 'geometry'], dtype='object')

Rename `'code_00'`, the column with classification.

In [10]:
clc2000 = clc2000.rename(columns={"code_00": "land_cover"})

Interpolate data to LSOA geometry.

In [11]:
%%time
lsoa_clc2000 = area_interpolate(clc2000, lsoa, categorical_variables=["land_cover"], n_jobs=15)

CPU times: user 5min 49s, sys: 8.96 s, total: 5min 58s
Wall time: 6min 46s


Save to a temporary parquet file.

In [13]:
lsoa_clc2000.to_parquet("../../data/clustering_phase_1/lsoa_clc2000.pq")

<ipython-input-13-880e04f11f1a>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  lsoa_clc2000.to_parquet("../../data/clustering_phase_1/lsoa_clc2000.pq")


### 2006

Check layers to make sure we're loading the correct one.

In [21]:
path2006 = "../../data/clc/2006/u2012_clc2006_v2020_20u1_geoPackage/DATA/U2012_CLC2006_V2020_20u1.gpkg"
lyr = fiona.listlayers(path2006)
lyr

['U2012_CLC2006_V2020_20u1',
 'U2012_CLC2006_V2020_20u1_FR_REU',
 'U2012_CLC2006_V2020_20u1_FR_GLP',
 'U2012_CLC2006_V2020_20u1_FR_GUF',
 'U2012_CLC2006_V2020_20u1_FR_MTQ',
 'U2012_CLC2006_V2020_20u1_FR_MYT']

Read data within the bounding box and reproject to CRS of LSOA.

In [22]:
clc2006 = gpd.read_file(path2006, bbox=bbox, layer="U2012_CLC2006_V2020_20u1")
clc2006 = clc2006.to_crs(lsoa.crs)
clc2006.columns

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:565: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Index(['Code_06', 'ID', 'Remark', 'Area_Ha', 'geometry'], dtype='object')

Rename `'Code_06'`, the column with classification.

In [23]:
clc2006 = clc2006.rename(columns={"Code_06": "land_cover"})

Interpolate data to LSOA geometry.Interpolate data to LSOA geometry.


In [25]:
%%time
lsoa_clc2006 = area_interpolate(clc2006, lsoa, categorical_variables=["land_cover"], n_jobs=15)

CPU times: user 7min 14s, sys: 8.96 s, total: 7min 23s
Wall time: 7min 49s


Save to a temporary parquet file.

In [26]:
lsoa_clc2006.to_parquet("../../data/clustering_phase_1/lsoa_clc2006.pq")

<ipython-input-26-7581f1724a52>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  lsoa_clc2006.to_parquet("../../data/clustering_phase_1/lsoa_clc2006.pq")


### 2012

Check layers to make sure we're loading the correct one.

In [27]:
path2012 = "../../data/clc/2012/u2018_clc2012_v2020_20u1_geoPackage/DATA/U2018_CLC2012_V2020_20u1.gpkg"
lyr = fiona.listlayers(path2012)
lyr

['U2018_CLC2012_V2020_20u1',
 'U2018_CLC2012_V2020_20u1_FR_REU',
 'U2018_CLC2012_V2020_20u1_FR_GLP',
 'U2018_CLC2012_V2020_20u1_FR_GUF',
 'U2018_CLC2012_V2020_20u1_FR_MTQ',
 'U2018_CLC2012_V2020_20u1_FR_MYT']

Read data within the bounding box and reproject to CRS of LSOA.

In [28]:
clc2012 = gpd.read_file(path2012, bbox=bbox)
clc2012 = clc2012.to_crs(lsoa.crs)
clc2012.columns

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:565: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Index(['Code_12', 'Remark', 'Area_Ha', 'ID', 'geometry'], dtype='object')

Rename `'Code_12'`, the column with classification.

In [29]:
clc2012 = clc2012.rename(columns={"Code_12": "land_cover"})

Interpolate data to LSOA geometry.


In [30]:
%%time
lsoa_clc2012 = area_interpolate(clc2012, lsoa, categorical_variables=["land_cover"], n_jobs=15)

CPU times: user 7min 23s, sys: 9.5 s, total: 7min 32s
Wall time: 7min 57s


Save to a temporary parquet file.


In [37]:
lsoa_clc2012.to_parquet("../../data/clustering_phase_1/lsoa_clc2012.pq")

<ipython-input-37-54616977a04e>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  lsoa_clc2012.to_parquet("../../data/clustering_phase_1/lsoa_clc2012.pq")


### 2018

Check layers to make sure we're loading the correct one.

In [31]:
path2018 = "../../data/clc/2018/u2018_clc2018_v2020_20u1_geoPackage/DATA/U2018_CLC2018_V2020_20u1.gpkg"
lyr = fiona.listlayers(path2018)
lyr

['U2018_CLC2018_V2020_20u1',
 'U2018_CLC2018_V2020_20u1_FR_REU',
 'U2018_CLC2018_V2020_20u1_FR_GLP',
 'U2018_CLC2018_V2020_20u1_FR_GUF',
 'U2018_CLC2018_V2020_20u1_FR_MTQ',
 'U2018_CLC2018_V2020_20u1_FR_MYT']

Read data within the bounding box and reproject to CRS of LSOA.


In [32]:
clc2018 = gpd.read_file(path2018, bbox=bbox)
clc2018 = clc2018.to_crs(lsoa.crs)
clc2018.columns

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:565: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Index(['Code_18', 'Remark', 'Area_Ha', 'ID', 'geometry'], dtype='object')

Rename `'Code_18'`, the column with classification.

In [33]:
clc2018 = clc2018.rename(columns={"Code_18": "land_cover"})

Interpolate data to LSOA geometry.


In [34]:
%%time
lsoa_clc2018 = area_interpolate(clc2018, lsoa, categorical_variables=["land_cover"], n_jobs=15)

CPU times: user 7min 28s, sys: 9.65 s, total: 7min 38s
Wall time: 8min 5s


Save to a temporary parquet file.

In [36]:
lsoa_clc2018.to_parquet("../../data/clustering_phase_1/lsoa_clc2018.pq")

<ipython-input-36-b29cc3faa52e>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  lsoa_clc2018.to_parquet("../../data/clustering_phase_1/lsoa_clc2018.pq")


## Create single table

In [36]:
lsoa_clc2000 = gpd.read_parquet("../../data/clustering_phase_1/lsoa_clc2000.pq")
lsoa_clc2006 = gpd.read_parquet("../../data/clustering_phase_1/lsoa_clc2006.pq")
lsoa_clc2012 = gpd.read_parquet("../../data/clustering_phase_1/lsoa_clc2012.pq")
lsoa_clc2018 = gpd.read_parquet("../../data/clustering_phase_1/lsoa_clc2018.pq")

In [28]:
legend = pd.read_csv("../../data/clc/2018/u2018_clc2018_v2020_20u1_geoPackage/Legend/CLC_legend.csv", sep=";")

In [29]:
legend = legend.set_index("CLC_CODE")

In [37]:
rename = {}
for p in [lsoa_clc2000, lsoa_clc2006, lsoa_clc2012, lsoa_clc2018]:
    for c in p.columns.drop('geometry'):
        rename[c] = legend.LABEL3[int(c[-3:])] + f" [{c[-3:]}]"

In [38]:
for p in [lsoa_clc2000, lsoa_clc2006, lsoa_clc2012, lsoa_clc2018]:
    p.rename(columns=rename, inplace=True)

In [39]:
for year, p in zip([2000, 2006, 2012, 2018], [lsoa_clc2000, lsoa_clc2006, lsoa_clc2012, lsoa_clc2018]):
    rename = {} 
    for c in p.columns.drop('geometry'):
        rename[c] = c + f" ({year})"
    p.rename(columns=rename, inplace=True)

In [41]:
complete = pd.concat([lsoa_clc2000.drop(columns='geometry'), lsoa_clc2006.drop(columns='geometry'), 
                      lsoa_clc2012.drop(columns='geometry'), lsoa_clc2018.drop(columns='geometry')], axis=1)

In [43]:
lsoa_path = "../../data/lsoa"
lsoa_inp = gpd.read_file(lsoa_path)

In [45]:
complete = pd.concat([lsoa_inp[['LSOA11CD', 'LSOA11NM']], complete], axis=1)

In [47]:
complete.to_csv("../../data/lsoa_land_cover.csv")

In [48]:
complete.head(5)

,LSOA11CD,LSOA11NM,Continuous urban fabric [111] (2000),Discontinuous urban fabric [112] (2000),Industrial or commercial units [121] (2000),Road and rail networks and associated land [122] (2000),Port areas [123] (2000),Airports [124] (2000),Mineral extraction sites [131] (2000),Dump sites [132] (2000),...,Road and rail networks and associated land [122] (2018),Water courses [511] (2018),Water bodies [512] (2018),Dump sites [132] (2018),Sparsely vegetated areas [333] (2018),Bare rocks [332] (2018),Sea and ocean [523] (2018),Burnt areas [334] (2018),Fruit trees and berry plantations [222] (2018),Agro-forestry areas [244] (2018)
0,E01000001,City of London 001A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,E01000002,City of London 001B,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E01000003,City of London 001C,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,E01000005,City of London 001E,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,E01000006,Barking and Dagenham 016A,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
